## Assignment 2 Part B Producer 1

**Name: Kang Hong Bo (Student ID:32684673)**

Write a python program that loads all the data from climate_streaming.csv and randomly (with replacement) feed the data to the stream every 10 seconds.  You will need to append additional information such as producer information to identify the producer and created date.

In [1]:
#install kakfa-python3
!pip3 install updates kafka-python3

In [2]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
from datetime import datetime,timedelta
ip="192.168.22.110" ## PLEASE CHANGE THIS LINE TO YOUR HOSTIP
client = MongoClient (ip,27017) 
db = client.fit3182_assignment_db
collection = db.partA

Read and publish the climate streaming data

In [3]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import datetime as dt

#read climate streaming data
def read_climate_streaming():
    c_s = pd.read_csv('climate_streaming.csv')
    #remove the extra space in the'precipitation '
    c_s.rename(columns = {'precipitation ':'precipitation'}, inplace = True)
    
    #splitting the precipitationFlag and the value of precipitation
    c_s['precipitationFlag']=c_s['precipitation'].str[-1]
    c_s['precipitation']=c_s['precipitation'].str[0:-1]
    
    read_data = []
    for index, row in c_s.iterrows():
        docs = {}
        docs['latitude']=float(row['latitude'])
        docs['longitude']=float(row['longitude'])
        docs['air_temperature_celcius']= int(row['air_temperature_celcius'])
        docs['relative_humidity']=float(row['relative_humidity'])
        docs['windspeed_knots']=float(row['windspeed_knots'])
        docs['max_wind_speed']=float(row['max_wind_speed'])
        docs['precipitation']=float(row['precipitation'].strip())
        docs['precipitationFlag'] = row['precipitationFlag'].strip()
        docs['GHI_w/m2']= int(row['GHI_w/m2'])
        read_data.append(docs)
    
    return read_data
# get the latest date in the collection
def get_latest_date():
    latest_date=collection.find({},{'_id':0,'date':1}).sort("date",pymongo.DESCENDING).limit(1)
    return latest_date[0]['date']

def publish_message(producer_instance, topic_name,key,data):
    try:
        
        producer_instance.send(topic_name, value=data)
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
    

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{ip}:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as e:
        print('Exception while connecting Kafka.')
        print(e)
    finally:
        return _producer
if __name__ == '__main__':
    
    topic = 'PartB'
    print("Publishing records...")
    producer01 = connect_kafka_producer()
    data = read_climate_streaming()
    daystoAdd=0
    latest_date=get_latest_date()+timedelta(days=1)
    
    
    while True:
        dataToPublish = random.choice(data)
        currentDate = latest_date + timedelta(days=daystoAdd)
        dataToPublish['producer']="climate_streaming"
        dataToPublish["createdDate"]=currentDate.strftime("%d/%m/%Y")
#         pprint(dataToPublish)
        publish_message(producer01,topic,"parse",dataToPublish)
        daystoAdd+=1
        sleep(10)

Publishing records...
Message published successfully. Data: {'latitude': -37.6008, 'longitude': 143.0558, 'air_temperature_celcius': 15, 'relative_humidity': 44.8, 'windspeed_knots': 8.3, 'max_wind_speed': 19.0, 'precipitation': 0.0, 'precipitationFlag': 'I', 'GHI_w/m2': 134, 'producer': 'climate_streaming', 'createdDate': '02/01/2024'}
Message published successfully. Data: {'latitude': -37.242, 'longitude': 141.153, 'air_temperature_celcius': 12, 'relative_humidity': 42.8, 'windspeed_knots': 12.1, 'max_wind_speed': 19.0, 'precipitation': 0.04, 'precipitationFlag': 'G', 'GHI_w/m2': 109, 'producer': 'climate_streaming', 'createdDate': '03/01/2024'}
Message published successfully. Data: {'latitude': -37.391, 'longitude': 148.066, 'air_temperature_celcius': 14, 'relative_humidity': 44.6, 'windspeed_knots': 7.7, 'max_wind_speed': 15.9, 'precipitation': 0.03, 'precipitationFlag': 'G', 'GHI_w/m2': 125, 'producer': 'climate_streaming', 'createdDate': '04/01/2024'}
Message published successful

KeyboardInterrupt: 